<a href="https://colab.research.google.com/github/jlopetegui98/NER-ClinicalTrials-Elegibility-Criteria/blob/main/LLM-Zero-shot_approach/generate_annotations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Annotations for the CHIA dataset using mistral-7B

In [1]:
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TextGenerationPipeline
import torch
import os
from utils import *

In [4]:
model_name = "mistralai/Mistral-7B-v0.1"

In [5]:
# Load base model(Mistral 7B)
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
   model_name,
    quantization_config=bnb_config,
    device_map={"": 0}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [6]:
# import tokenizer for mistral-7B
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.padding_side = 'left'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

(True, True)

In [8]:
pipe  = TextGenerationPipeline(model = model, tokenizer = tokenizer)

In [9]:
# load dataset for annotation (To make the model better for generalization we can generate a class that receive
# the dataset in BIO format as input and make all parsing process and annotations)
data_path = "./drive/MyDrive/HandsOn-NLP/data/chia_criteria_texts/"

In [12]:
# get the list of files in the dataset
files = os.listdir(data_path)
files[:5]

['NCT03264911_inc.bio.txt',
 'NCT03134378_inc.bio.txt',
 'NCT02924090_exc.bio.txt',
 'NCT02555163_inc.bio.txt',
 'NCT02952378_exc.bio.txt']

In [20]:
# example of input using first prompt
with open(f"{data_path}{files[1]}", 'r+') as fd:
  first_sentence = fd.readlines()[0]
print(f"Source sentence: {first_sentence}")
print("First prompt:")
print(build_prompt(first_sentence, 1))

Source sentence: 18 years or older patients who are proven to be infected by Helicobacter pylori based on positive in Urea Breath Test or positive in histopathologic examination of biopsy in antrum and corpus of gaster through esophagoduodenoscopy .

First prompt:
I need to perform a named entity recognition task on a  text related with inclusion criteria in clinical trials.
    The entities you need to recognize are: Condition, Value, Drug, Procedure, Measurement, Temporal, Observation, Person, Mood, Device and Pregnancy_considerations.
    Particularly you have to produce the ouput in the BIO format. I will show you an example of the expected output.
    Input text: Patients with symptomatic CNS metastases or leptomeningeal involvement
    Output:
    Patients O
    with O
    symptomatic O
    CNS B-Condition
    metastases I-Condition
    or O
    leptomeningeal B-Condition
    involvement I-Condition

    You can see that tokens without any entity are labeled as O, and the tokens 

In [21]:
print("Second prompt:")
print(build_prompt(first_sentence, 2))

Second prompt:
I am working on a named entity recognition problem, in the context of clinical
    trials eligibility criteria. I will show you the list of entities:
    - Condition
    - Value
    - Drug
    - Procedure
    - Measurement
    - Temporal
    - Observation
    - Person
    - Mood
    - Device

    Your task consists in annotate the named entities in a given sentence in the format I will explain you.
    I will explain you with some examples:

    Example 1:
    Input: Patients who have received prior chemotherapy for unresectable disease.
    Output: Patients who have received prior <Procedure>chemotherapy</Procedure> for <Condition>unresectable disease</Condition>.

    Example 2:
    Input: Patients with any other severe concurrent disease, which in the judgment of the investigator, would make the patient inappropriate for entry into this study.
    Ouput: Patients with any other severe <Condition>concurrent disease</Condition>, which in the judgment of the investigator

In [27]:
from tqdm import tqdm

In [32]:
# generate annotations for first prompt
for file in tqdm(files[:20]):
  sentences = []
  with open(f"{data_path}{file}", 'r+', encoding='utf-8') as fd:
    sentences = fd.readlines()
  annotated_sentences = []
  for sentence in sentences:
    if len(sentence) == 0:
      continue
    input = build_prompt(sentence, 1)
    output = pipe(input, max_new_tokens = 500, return_full_text = False, handle_long_generation = "hole")[0]['generated_text']
    output = output.split('\n\n')[0]#to get just the first sentence in the ouput and avoid noise
    annotated_sentences.append(output)
  with open(f"./drive/MyDrive/HandsOn-NLP/data/Annotations_Mistral_Prompt_1/{file}", 'w+', encoding='utf-8') as fd:
    fd.write('\n\n'.join(annotated_sentences))

  0%|          | 0/6 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
 17%|█▋        | 1/6 [00:42<03:34, 42.80s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
 33%|███▎      | 2/6 [02:06<04:28, 67.01s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
 50%|█████     | 3/6 [06:16<07:31, 150.43s/it

In [40]:
# generate annotations for second prompt
for file in tqdm(files[:20]):
  print(file)
  sentences = []
  with open(f"{data_path}{file}", 'r+', encoding='utf-8') as fd:
    sentences = fd.readlines()
  annotated_sentences = []
  for sentence in sentences:
    if len(sentence) == 0:
      continue
    input = build_prompt(sentence, 2)
    input += '\noutput: '
    output = pipe(input, max_new_tokens = 500, return_full_text = False, handle_long_generation = "hole")[0]['generated_text']
    output = output.split('\n')[0] + '.' #to get just the first sentence in the ouput and avoid noise
    annotated_sentences.append(output)
  with open(f"./drive/MyDrive/HandsOn-NLP/data/Annotations_Mistral_Prompt_2/{file}", 'w+', encoding='utf-8') as fd:
    fd.write('\n\n'.join(annotated_sentences))

  0%|          | 0/20 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


NCT03264911_inc.bio.txt


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  5%|▌         | 1/20 [02:48<53:28, 168.89s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


NCT03134378_inc.bio.txt


 10%|█         | 2/20 [03:30<28:15, 94.20s/it] Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
 90%|█████████ | 18/20 [48:46<05:31, 165.99s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


NCT02894372_inc.bio.txt


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
 95%|█████████▌| 19/20 [50:08<02:21, 141.01s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


NCT00730301_inc.bio.txt


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
100%|██████████| 20/20 [59:05<00:00, 177.28s/it]
